Ниже приведены **два** решения задачи.

**Первое** использует модель **Grounding DINO**, которая находит спортсмена по **текстовому описанию**. Модель работает с минимальным количеством ложных детекций и демонстрирует **стабильно точные результаты**, однако каждый вызов этой модели длится дольше, чем более легковесные аналоги вроде YOLO.

**Второе** решение является **оптимизированной версией** первого решения. В нем **минимизируется количество вызовов** модели Grounding DINO. Эта модель вызывается всего 2 раза в секунду. Bounding boxes этой модели являются опорой для **более быстрой модели** YOLO (или любой аналогичной модели).

Этот код реализует систему автоматического **обнаружения, отслеживания и оценки позы спортсмена на видео**.

Он использует модель **Grounding DINO** для детекции по текстовому описанию с частотой 5 раз в секунду.
Затем активируется **трекер CSRT**, поддерживающий отслеживание спортсмена в промежутках **между детекциями**.

Для повышения скорости детекция выполняется на **уменьшенной копии кадра**, а координаты масштабируются обратно.

Если объект **успешно детектирован** или оттрекан, производится **обрезка** соответствующей **области**, на которую затем применяется модель YOLO **для оценки позы**.

Полученные ключевые точки отображаются на оригинальном кадре в **виде красных точек**.

Готовое видео записывается **в выходной файл**. Также в коде учитывается возможность работы на GPU с использованием смешанной точности для ускорения вычислений.

In [ ]:
!pip install ultralytics
!pip install deep-sort-realtime
!pip install openmim
!mim install mmdet
!mim install mmpose

# Grounding Dino
!git clone https://github.com/IDEA-Research/GroundingDINO.git
!cd GroundingDINO
!pip install --upgrade pip setuptools wheel cython
!pip install -r requirements.txt
!pip install pycocotools supervision yapf addict
!pip install -e . --no-build-isolation



In [ ]:
import cv2
import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForZeroShotObjectDetection
from ultralytics import YOLO
import time

start_execution_time = time.time()

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_id = "IDEA-Research/grounding-dino-tiny"
processor = AutoProcessor.from_pretrained(model_id)
model = AutoModelForZeroShotObjectDetection.from_pretrained(model_id).to(device)

pose_model = YOLO('yolo11m-pose.pt')

# Mixed precision
use_amp = device == 'cuda'
if use_amp:
    model = model.half()

cap = cv2.VideoCapture('/content/0LtLS9wROrk_E_000731_000738.mp4')
fps = cap.get(cv2.CAP_PROP_FPS)
w, h = int(cap.get(3)), int(cap.get(4))
out = cv2.VideoWriter('gdino_out_1.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))

# Parameters
DETECTION_FPS = 5  # раз в секунду
FRAME_SKIP = int(fps / DETECTION_FPS)
SCALE_FACTOR = 1/10
prompt = [["a gymnast", "a gymnast upside down"]]

frame_count = 0
last_detection = None
tracker = None
tracker_active = False

small_w, small_h = int(w * SCALE_FACTOR), int(h * SCALE_FACTOR)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    current_detection = None

    if frame_count % FRAME_SKIP == 0:
        small_frame = cv2.resize(frame, (small_w, small_h))
        img = Image.fromarray(cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB))
        inputs = processor(images=img, text=prompt, return_tensors="pt")

        if use_amp:
            inputs = {k: v.to(device).half() if v.dtype == torch.float32 else v.to(device)
                      for k, v in inputs.items()}
        else:
            inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
            if use_amp:
                with torch.cuda.amp.autocast():
                    outputs = model(**inputs)
            else:
                outputs = model(**inputs)

        results = processor.post_process_grounded_object_detection(
            outputs,
            inputs['input_ids'],
            box_threshold=0.3,
            text_threshold=0.25,
            target_sizes=[(small_h, small_w)]
        )

        boxes = results[0]["boxes"]
        scores = results[0]["scores"]
        labels = results[0]["labels"]

        if len(scores) > 0:
            max_score_idx = torch.argmax(scores).item()
            best_box = boxes[max_score_idx].tolist()
            best_score = scores[max_score_idx].item()
            best_label = labels[max_score_idx]

            scale_x = w / small_w
            scale_y = h / small_h
            best_box = [
                int(best_box[0] * scale_x),
                int(best_box[1] * scale_y),
                int(best_box[2] * scale_x),
                int(best_box[3] * scale_y)
            ]

            current_detection = {
                'box': best_box,
                'score': best_score,
                'label': best_label
            }
            last_detection = current_detection

            tracker = cv2.TrackerCSRT_create()
            x1, y1, x2, y2 = best_box
            tracker.init(frame, (x1, y1, x2 - x1, y2 - y1))
            tracker_active = True

    else:
        if tracker_active and tracker is not None:
            success, tracked_box = tracker.update(frame)
            if success:
                x, y, w_box, h_box = map(int, tracked_box)
                tracked_detection = {
                    'box': [x, y, x + w_box, y + h_box],
                    'score': None,
                    'label': 'tracked'
                }
                last_detection = tracked_detection
            else:
                tracker_active = False
                tracker = None

    detection_to_draw = current_detection or last_detection

    if detection_to_draw:
        box = detection_to_draw['box']
        score = detection_to_draw['score']
        label = detection_to_draw.get('label', '')

        # draw box
        cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)
        if score is not None:
            cv2.putText(frame, f"{label} {score:.2f}", (box[0], box[1] - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        else:
            cv2.putText(frame, f"{label}", (box[0], box[1] - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)

        # Pose estimation
        cropped = frame[box[1]:box[3], box[0]:box[2]]
        if cropped.size > 0:
            pose_results = pose_model.predict(cropped, imgsz=256, conf=0.3, verbose=False)
            for kp in pose_results[0].keypoints.xy:
                for x, y in kp:
                    cx = int(x.item() + box[0])
                    cy = int(y.item() + box[1])
                    cv2.circle(frame, (cx, cy), 3, (0, 0, 255), -1)

    out.write(frame)
    frame_count += 1

cap.release()
out.release()

end_execution_time = time.time()
print("Время выполнения:", end_execution_time - start_execution_time)


Данный код является оптимизированной версией прошлого решения.

В процессе работы видео последовательно считывается покадрово, и на каждом кадре производится **либо первичная детекция** с помощью модели **Grounding DINO**, **либо уточнение** и обновление локализации спортсмена с помощью модели YOLO (или любой другой легковесной модели).

**Grounding DINO** запускается **с низкой частотой** и выполняет обнаружение объектов на основе текстовых описаний, при этом обработка происходит на уменьшенной копии кадра для ускорения работы. Найденные координаты масштабируются обратно к исходному размеру изображения.

В промежутках **между вызовами DINO**, модель **YOLO** применяется для **дообнаружения** или **уточнения** позиции спортсмена **в ограниченной области** вокруг **предыдущей детекции**.

Эта **область расширяется** по горизонтали и вертикали на заданное значение, чтобы обеспечить **более устойчивое обнаружение**.

Если YOLO успешно находит человека, его координаты сохраняются и используются **для последующей обработки**.

После того как регион спортсмена определён, он обрезается, и на него запускается **модель оценки поз YOLO Pose**, которая определяет **ключевые точки тела**.

Эти ключевые точки наносятся на исходный кадр в виде красных точек. Обработанные кадры записываются **в выходное видео**.

In [ ]:
import cv2
import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForZeroShotObjectDetection
from ultralytics import YOLO
import time

start_execution_time = time.time()

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_id = "IDEA-Research/grounding-dino-tiny"
processor = AutoProcessor.from_pretrained(model_id)
model = AutoModelForZeroShotObjectDetection.from_pretrained(model_id).to(device)

yolo_model = YOLO("yolo11l.pt")

pose_model = YOLO('yolo11m-pose.pt')

# Mixed precision if available
use_amp = device == 'cuda'
if use_amp:
    model = model.half()

cap = cv2.VideoCapture('/content/0LtLS9wROrk_E_000731_000738.mp4')
fps = cap.get(cv2.CAP_PROP_FPS)
w, h = int(cap.get(3)), int(cap.get(4))
out = cv2.VideoWriter('gdino_out_2.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))


# optimization parameters
GROUNDING_DINO_FPS = 2
GROUNDING_DINO_FRAME_SKIP = fps // GROUNDING_DINO_FPS

YOLO_FPS = 7
YOLO_FRAME_SKIP = fps // YOLO_FPS

SCALE_FACTOR = 1/10
EXPANSION_RATIO = 0.4


prompt = [["a gymnast", "a gymnast upside down"]] # Grounding DINO prompts
frame_count = 0
last_detection = None

small_w, small_h = int(w * SCALE_FACTOR), int(h * SCALE_FACTOR)

def expand_box(box, img_w, img_h, ratio_hor, ratio_ver):
    x1, y1, x2, y2 = box
    w_box = x2 - x1
    h_box = y2 - y1
    pad_x = int(w_box * ratio_hor / 2)
    pad_y = int(h_box * ratio_ver / 2)
    x1_new = max(0, x1 - pad_x)
    y1_new = max(0, y1 - pad_y)
    x2_new = min(img_w, x2 + pad_x)
    y2_new = min(img_h, y2 + pad_y)
    return [x1_new, y1_new, x2_new, y2_new]


while True:
    ret, frame = cap.read()
    if not ret:
        break

    current_detection = None

    if frame_count % GROUNDING_DINO_FRAME_SKIP == 0:
        # resizing image
        small_frame = cv2.resize(frame, (small_w, small_h))
        img = Image.fromarray(cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB))

        inputs = processor(images=img, text=prompt, return_tensors="pt")

        # mixed precision
        if use_amp:
            inputs = {k: v.to(device).half() if v.dtype == torch.float32 else v.to(device)
                     for k, v in inputs.items()}
        else:
            inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
            if use_amp:
                with torch.cuda.amp.autocast():
                    outputs = model(**inputs)
            else:
                outputs = model(**inputs)

        results = processor.post_process_grounded_object_detection(
            outputs,
            inputs['input_ids'],
            box_threshold=0.3,
            text_threshold=0.25,
            target_sizes=[(small_h, small_w)]
        )

        boxes = results[0]["boxes"]
        scores = results[0]["scores"]
        labels = results[0]["labels"]

        if len(scores) > 0:
            # get index with the highest confidence
            max_score_idx = torch.argmax(scores).item()

            best_box = boxes[max_score_idx].tolist()
            best_score = scores[max_score_idx].item()
            best_label = labels[max_score_idx]

            # rescaling image
            scale_x = w / small_w
            scale_y = h / small_h
            best_box = [
                int(best_box[0] * scale_x),
                int(best_box[1] * scale_y),
                int(best_box[2] * scale_x),
                int(best_box[3] * scale_y)
            ]

            current_detection = {
                'box': best_box,
                'score': best_score,
                'label': best_label
            }
            last_detection = current_detection

    elif (frame_count % YOLO_FRAME_SKIP == 0) and (frame_count % GROUNDING_DINO_FRAME_SKIP != 0):
        area_to_detect = current_detection or last_detection
        area_to_detect = area_to_detect["box"]
        area_to_detect = expand_box(area_to_detect, w, h, EXPANSION_RATIO, EXPANSION_RATIO + 0.3)

        x1, y1, x2, y2 = area_to_detect
        cropped_frame = frame[y1:y2, x1:x2]
        results = yolo_model(cropped_frame, verbose=False)

        detections = results[0].boxes
        if detections is not None and detections.xyxy.shape[0] > 0:
            boxes = detections.xyxy
            scores = detections.conf
            classes = detections.cls

            person_indices = (classes == 0).nonzero(as_tuple=True)[0]
            if len(person_indices) > 0:
                # get the best prediction
                best_idx = person_indices[scores[person_indices].argmax().item()]
                box = boxes[best_idx].tolist()
                score = scores[best_idx].item()
                label = int(classes[best_idx].item())

                current_detection = {
                    'box': [int(box[0] + x1), int(box[1] + y1), int(box[2] + x1), int(box[3] + y1)],
                    'score': score,
                    'label': label
                }
                last_detection = current_detection

    detection_to_draw = current_detection or last_detection

    if detection_to_draw:
        box = detection_to_draw['box']
        score = detection_to_draw['score']

        # Draw bounding box
        cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)
        cv2.putText(frame, f"{prompt[0]} {score:.2f}", (box[0], box[1]-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        # Crop the person region for pose estimation
        cropped = frame[box[1]:box[3], box[0]:box[2]]
        if cropped.size > 0:
            pose_results = pose_model.predict(cropped, imgsz=256, conf=0.3, verbose=False)

            # Draw keypoints on the original frame (rescale points to original image)
            for kp in pose_results[0].keypoints.xy:
                for x, y in kp:
                    cx = int(x.item() + box[0])
                    cy = int(y.item() + box[1])
                    cv2.circle(frame, (cx, cy), 3, (0, 0, 255), -1)

    out.write(frame)
    frame_count += 1

cap.release()
out.release()

end_execution_time = time.time()
print("Время выполнения:", end_execution_time - start_execution_time)